# MongoDB 与 Python 的结合教程

## pymongo库
- 安装 `pymongo` 库
- 安装其他可选库（如 `folium`）


In [1]:
!pip install pymongo folium


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 数据准备
- 示例数据集介绍（如餐馆和社区数据）
- 数据集下载链接
- 数据导入 MongoDB 的方法

In [ ]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

# 示例 GeoJSON 数据集下载链接
earthquakes_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson'
people_url = 'https://datahub.io/core/geo-countries/r/countries.geojson'
# 加载 earthquakes GeoJSON 数据
earthquakes_response = requests.get(earthquakes_url)
earthquakes_data = earthquakes_response.json()

# 插入 earthquakes 数据到 MongoDB
db.earthquakes.insert_many(earthquakes_data['features'])
print(f"已导入 {len(earthquakes_data['features'])} 条地震数据到 MongoDB。")

# 加载 people GeoJSON 数据
people_response = requests.get(people_url)
people_data = people_response.json()

# 插入 people 数据到 MongoDB
db.people.insert_many(people_data['features'])
print(f"已导入 {len(people_data['features'])} 条人口数据到 MongoDB。")


## 统计每个国家地震的数量

In [4]:
from pymongo import MongoClient

# 连接到 MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['geo_demo']
collection = db['earthquakes']

# 聚合查询提取国家或地区
pipeline = [
    {"$project": {"place": "$properties.place"}},  # 提取 place 字段
    {"$group": {
        "_id": {"$arrayElemAt": [{"$split": ["$place", ", "]}, -1]},  # 获取最后一部分（国家或地区）
        "count": {"$sum": 1}  # 统计每个国家或地区的地震记录数量
    }},
    {"$sort": {"count": -1}}  # 按地震数量降序排序
]

# 执行聚合查询并打印结果
results = collection.aggregate(pipeline)
for result in results:
    print(f"Country/Region: {result['_id']}, Earthquake Count: {result['count']}")

Country/Region: CA, Earthquake Count: 1366
Country/Region: Alaska, Earthquake Count: 960
Country/Region: Oklahoma, Earthquake Count: 208
Country/Region: Nevada, Earthquake Count: 128
Country/Region: Texas, Earthquake Count: 122
Country/Region: Montana, Earthquake Count: 114
Country/Region: Puerto Rico, Earthquake Count: 112
Country/Region: Washington, Earthquake Count: 99
Country/Region: Hawaii, Earthquake Count: 56
Country/Region: New Mexico, Earthquake Count: 54
Country/Region: Idaho, Earthquake Count: 52
Country/Region: Utah, Earthquake Count: 32
Country/Region: Oregon, Earthquake Count: 26
Country/Region: California, Earthquake Count: 24
Country/Region: Indonesia, Earthquake Count: 22
Country/Region: U.S. Virgin Islands, Earthquake Count: 20
Country/Region: Canada, Earthquake Count: 18
Country/Region: Japan, Earthquake Count: 18
Country/Region: Wyoming, Earthquake Count: 16
Country/Region: Russia, Earthquake Count: 16
Country/Region: Tonga, Earthquake Count: 14
Country/Region: Peru

## 3. MongoDB 基础操作
### 3.1 数据库与集合
- 创建数据库和集合
- 查看现有数据库和集合


In [ ]:
# 查看现有数据库
print("现有数据库：", client.list_database_names())

现有数据库： ['admin', 'config', 'geo_demo', 'local']


In [ ]:
# 查看当前数据库中的集合
print("当前数据库中的集合：", db.list_collection_names())

当前数据库中的集合： ['earthquakes', 'people', 'test_collection']


### 3.2 基本 CRUD 操作
- 插入文档
- 查询文档
- 更新文档
- 删除文档


In [7]:
# 插入文档
doc = {"name": "Alice", "age": 28, "city": "Beijing"}
insert_result = db.test_collection.insert_one(doc)
print("插入文档ID:", insert_result.inserted_id)


插入文档ID: 682a96759d453b2e55b23940


In [8]:
# 查询文档
found_doc = db.test_collection.find_one({"name": "Alice"})
print("查询结果:", found_doc)


查询结果: {'_id': ObjectId('682a96759d453b2e55b23940'), 'name': 'Alice', 'age': 28, 'city': 'Beijing'}


In [9]:
# 更新文档
update_result = db.test_collection.update_one({"name": "Alice"}, {"$set": {"age": 29}})
print("更新的文档数:", update_result.modified_count)


更新的文档数: 1


In [10]:
# 删除文档
delete_result = db.test_collection.delete_one({"name": "Alice"})
print("删除的文档数:", delete_result.deleted_count)

删除的文档数: 1


## 4. Python 与 MongoDB 的结合
### 4.1 使用 `pymongo` 连接 MongoDB
- 连接本地 MongoDB
- 连接远程 MongoDB Atlas

### 4.2 数据库与集合操作
- 切换数据库
- 切换集合

### 4.3 数据插入与查询
- 插入单条和多条数据
- 查询数据（条件查询、投影）

### 4.4 数据更新与删除
- 更新单条和多条数据
- 删除单条和多条数据

## 5. 地理空间查询
### 5.1 地理空间索引
- 什么是地理空间索引？
- 创建 `2dsphere` 索引


In [ ]:
# 为 earthquakes 集合创建 2dsphere 索引
db.earthquakes.create_index([("geometry", "2dsphere")])

'geometry_2dsphere'

In [ ]:
# 为 people 集合创建 2dsphere 索引
db.people.create_index([("geometry", "2dsphere")])

'geometry_2dsphere'

### 5.2 地理空间查询操作符
- `$geoWithin`：查询区域内的点
- `$geoIntersects`：查询与区域相交的点
- `$nearSphere`：查询距离某点最近的点


In [16]:
# 查询日本区域内发生的地震（$geoWithin）
# 从 people_data 中提取日本的区域坐标
# 查询日本的国界（Polygon）
# 查找people 集合中日本的国界
japan_feature = db.people.find_one({"properties.name": "Japan"})
japan_polygon = japan_feature['geometry']
japan_earthquakes = list(db.earthquakes.find({
        "geometry": {
            "$geoWithin": {
                "$geometry": japan_polygon
            }
        }
    }))
print(f"日本区域内地震数量: {len(japan_earthquakes)}")

日本区域内地震数量: 6


In [15]:
# 题目2：查询距离北京最近的地震（$nearSphere）
tiananmen_point = {
    "type": "Point",
    "coordinates": [116.397389, 39.908722]
}
nearest_earthquake = db.earthquakes.find_one({
    "geometry": {
        "$nearSphere": {
            "$geometry": tiananmen_point
        }
    }
})
print("距离天安门最近的地震信息:", nearest_earthquake)

距离天安门最近的地震信息: {'_id': ObjectId('682757ba5f8b596c851cd4d2'), 'type': 'Feature', 'properties': {'mag': 4.7, 'place': '83 km SW of Kurio, Japan', 'time': 1746877515425, 'updated': 1746878586040, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000pyev', 'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/us7000pyev.geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'reviewed', 'tsunami': 0, 'sig': 340, 'net': 'us', 'code': '7000pyev', 'ids': ',us7000pyev,', 'sources': ',us,', 'types': ',origin,phase-data,', 'nst': 53, 'dmin': 1.882, 'rms': 0.83, 'gap': 119, 'magType': 'mb', 'type': 'earthquake', 'title': 'M 4.7 - 83 km SW of Kurio, Japan'}, 'geometry': {'type': 'Point', 'coordinates': [129.8769, 29.6884, 70.944]}, 'id': 'us7000pyev'}


In [2]:
from pymongo import MongoClient, GEOSPHERE
import plotly.express as px
import pandas as pd


In [3]:

# 连接到 MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['geo_demo']
# 创建地理空间索引（仅需运行一次）
db.earthquakes.create_index([("location", GEOSPHERE)])
db.countries.create_index([("borders", GEOSPHERE)])

# 创建时间索引（便于按时间查询）
result = db.earthquakes.create_index("date")
print("时间索引创建结果:", result)
print("当前索引列表:", list(db.earthquakes.index_information().keys()))


时间索引创建结果: date_1
当前索引列表: ['_id_', 'geometry_2dsphere', 'location_2dsphere', 'date_1']


# python联用的作业

# 任务书：MongoDB与Python地震数据可视化

## 一、任务目标

利用MongoDB数据库和Python编程，实现对全球地震数据的存储、查询与可视化。通过本任务，掌握MongoDB的基本操作、地理空间数据的处理方法，以及如何用Python进行数据可视化。

## 二、任务要求

1. **数据准备**
   - 下载全球地震数据（GeoJSON格式），并导入MongoDB数据库的`earthquakes`集合。
   - 熟悉MongoDB的基本操作，包括数据插入、查询等。

2. **数据查询**
   - 使用`pymongo`库从MongoDB中查询地震的经度、纬度、地点、时间和震级等信息。

3. **数据可视化**
   - 使用`plotly`库，将查询到的地震数据在世界地图上进行可视化展示。
   - 鼠标悬停时能显示地震的地点、时间和震级等详细信息。

## 三、任务步骤

1. **环境准备**
   - 安装MongoDB数据库，并启动服务。
   - 安装Python及相关库：`pymongo`、`pandas`、`plotly`。

2. **数据导入**
   - 下载
   [地震数据](https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson)
   
   - 使用Python将数据导入MongoDB的`earthquakes`集合。

3. **数据查询与处理**
   - 用`pymongo`查询地震的经纬度、地点、时间、震级等字段。
   - 用`pandas`整理为DataFrame格式，便于后续可视化。

4. **数据可视化**
   - 用`plotly.express.scatter_geo`在世界地图上绘制地震分布图。
   - 设置点的颜色和大小反映震级，鼠标悬停显示详细信息。

5. **结果展示**
   - 在Jupyter Notebook或Python脚本中展示可视化结果。

## 四、提交内容

1. 代码文件（.ipynb），要求有详细注释，以学号重命名，发邮箱`qqwqy@163.com`。
2. 运行结果（包含地图可视化效果）。


In [1]:
import pandas as pd
from pymongo import MongoClient
import plotly.express as px
import datetime

# 连接MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['geo_demo']
collection = db['earthquakes']

# 查询地震数据（只取部分字段，限制数量可加 limit）
cursor = collection.find({}, {
    'geometry.coordinates': 1,
    'properties.place': 1,
    'properties.time': 1,
    'properties.mag': 1
})

# 整理为DataFrame
records = []
for doc in cursor:
    coords = doc['geometry']['coordinates']
    lon, lat = coords[0], coords[1]
    mag = doc['properties'].get('mag', None)
    place = doc['properties'].get('place', '')
    time = doc['properties'].get('time', 0)
    # 时间戳转字符串
    time_str = datetime.datetime.utcfromtimestamp(time/1000).strftime('%Y-%m-%d %H:%M:%S')
    records.append({
        '经度': lon,
        '纬度': lat,
        '地点': place,
        '时间': time_str,
        '震级': mag
    })

df = pd.DataFrame(records)

# 用plotly画世界地图
# 震级小于0的取绝对值，并设置最小marker size，避免负值和0
df['marker_size'] = df['震级'].abs().clip(lower=0.1) * 5  # 可调整乘数以改变点大小

fig = px.scatter_geo(
    df,
    lon='经度',
    lat='纬度',
    hover_name='地点',
    hover_data=['时间', '震级'],
    color='震级',
    color_continuous_scale='Reds',
    size='marker_size',
    projection='natural earth',
    title='全球地震分布'
)
fig.show()